### This notebook first synthesizes synthetic data for a process with multiple marks. Afterwards it uses the expectation maximization algorithm of tick to approximate the kernel functions using the "bulk strategy"

In [1]:
%matplotlib inline

In [2]:
import copy
import itertools
import numpy as np
import matplotlib.pyplot as plt

from tick.plot import plot_basis_kernels, plot_hawkes_kernels
from tick.hawkes import SimuHawkes, HawkesKernelTimeFunc, HawkesBasisKernels, HawkesEM

In [3]:
num_marks = 3
end_time = 5e4
C = 1e-3
kernel_size = 100
max_iter = 100

mus = [1, 0.2, 0.5, 0.8]

# def g1(t):
#     alpha = 0.8
#     beta = 1.0
#     return alpha * beta * np.exp(-beta * t)

# def g2(t):
#     alpha_1 = 0.4
#     beta_1 = 1.0
#     alpha_2 = 0.4
#     beta_2 = 20
#     return alpha_1 * beta_1 * np.exp(-beta_1 * t) + alpha_2 * beta_2 * np.exp(-beta_2 * t)

# def g3(t):
#     alpha = 0.8
#     beta = 1.0
#     return alpha * beta * np.exp(-beta * t**2)

# def g4(t):
#     alpha = 0.4
#     beta = 5.0
#     return alpha * beta * np.exp(-beta * t**2)

def g1(t):
    alpha = 0.1
    beta = 0.5
    return np.maximum(0, alpha * beta * np.sin(beta * t))

def g2(t):
    alpha_1 = 0.02
    beta_1 = 0.5
    alpha_2 = 0.03
    beta_2 = 1.0
    return np.maximum(0, alpha_1 * beta_1 * np.exp(-beta_1 * t) + alpha_2 * beta_2 * np.exp(-beta_2 * t))

def g3(t):
    alpha = 0.1
    beta = 0.1
    return np.maximum(0, alpha * beta * np.exp(-beta * t))

def g4(t):
    alpha = 0.01
    beta = 1.0
    return np.maximum(0, alpha * beta * (1 - np.exp(-beta * t)))

t_values = np.linspace(0, 20, 1000)

In [4]:
function_values = [g1(t_values), g2(t_values), g3(t_values), g4(t_values)]

In [5]:
hawkes = SimuHawkes(baseline=mus, seed=1093, verbose=False)
for i in range(len(mus)):
    kernel = HawkesKernelTimeFunc(t_values=t_values, y_values=function_values[i])
    hawkes.set_kernel(i, i, kernel)

hawkes.end_time = end_time
hawkes.simulate()
ticks = hawkes.timestamps

In [6]:
for i in range(len(ticks)):
    print(f"Number of events for mark {i}: {len(ticks[i])}")

Number of events for mark 0: 83015
Number of events for mark 1: 10562
Number of events for mark 2: 27407
Number of events for mark 3: 49331


In [7]:
def get_bulked_ticks(ticks, selected_event_type):
    """
    Return ticks for the selected event type and the bulked ticks.
    """
    # Merge all event types except the selected one into the bulk
    bulked_ticks = np.concatenate([ticks[i] for i in range(len(ticks)) if i != selected_event_type])
    # Sort the bulked ticks
    bulked_ticks = np.sort(bulked_ticks)
    return [ticks[selected_event_type], bulked_ticks]

Plot the kernel with the bulking approach

In [8]:
mark = 0

bulked_ticks = get_bulked_ticks(ticks, mark)

def selected_kernel(t):
    return g1(t)

def bulk(t):
    return g2(t) + g3(t) + g4(t)

bulked_mus = [mus[mark], mus[1] + mus[2] + mus[3]]

bulked_hawkes = SimuHawkes(baseline=bulked_mus, seed=1093, verbose=False)
bulked_hawkes.set_kernel(0, 0, HawkesKernelTimeFunc(t_values=t_values, y_values=selected_kernel(t_values)))
bulked_hawkes.set_kernel(1, 1, HawkesKernelTimeFunc(t_values=t_values, y_values=bulk(t_values)))

bulked_hawkes.set_timestamps(bulked_ticks)

# And then perform estimation using expectation maximization with two basis kernels
# kernel_support = 20
# n_basis = 10

# em = HawkesBasisKernels(kernel_support, n_basis=n_basis,
#                         kernel_size=kernel_size, C=C, n_threads=12,
#                         max_iter=max_iter, verbose=False, ode_tol=1e-5)
em = HawkesEM(40, kernel_size=100, n_threads=8, verbose=False, tol=1e-3)
em.fit(bulked_hawkes.timestamps)

fig = plot_hawkes_kernels(em, hawkes=bulked_hawkes, show=False)
fig.set_size_inches(15, 10)  # Adjust the size as needed
plt.tight_layout()  # Adjust the layout to prevent overlap
for ax in fig.axes:
    ax.set_ylim([0, 0.1])

plt.show()
plt.savefig("bulk1.png")

In [9]:
mark = 1

bulked_ticks = get_bulked_ticks(ticks, mark)

def selected_kernel(t):
    return g2(t)

def bulk(t):
    return g1(t) + g3(t) + g4(t)

bulked_mus = [mus[mark], mus[0] + mus[2] + mus[3]]

bulked_hawkes = SimuHawkes(baseline=bulked_mus, seed=1093, verbose=False)
bulked_hawkes.set_kernel(0, 0, HawkesKernelTimeFunc(t_values=t_values, y_values=selected_kernel(t_values)))
bulked_hawkes.set_kernel(1, 1, HawkesKernelTimeFunc(t_values=t_values, y_values=bulk(t_values)))

bulked_hawkes.set_timestamps(bulked_ticks)

# And then perform estimation using expectation maximization with two basis kernels
em = HawkesEM(40, kernel_size=100, n_threads=8, verbose=False, tol=1e-3)
em.fit(bulked_hawkes.timestamps)

fig = plot_hawkes_kernels(em, hawkes=bulked_hawkes, show=False)
fig.set_size_inches(15, 10)  # Adjust the size as needed
plt.tight_layout()  # Adjust the layout to prevent overlap
for ax in fig.axes:
    ax.set_ylim([0, 0.1])

plt.show()
plt.savefig("bulk2.png")


In [10]:
# mark = 1

# bulked_ticks = get_bulked_ticks(ticks, mark)
# # And then perform estimation using expectation maximization with two basis kernels
# kernel_support = 20

# em = HawkesBasisKernels(kernel_support, n_basis=2,
#                         kernel_size=kernel_size, C=C, n_threads=12,
#                         max_iter=max_iter, verbose=False, ode_tol=1e-5)
# em.fit(bulked_ticks)

# def selected_kernel(t):
#     return g2(t)

# def bulk(t):
#     return g1(t) + g3(t) + g4(t)

# fig = plot_hawkes_kernels(em, show=False)
# for ax in fig.axes:
#     ax.set_ylim([0.0, 1])
    
# fig = plot_basis_kernels(em, basis_kernels=[g2, g1], show=False)
# for ax in fig.axes:
#     ax.set_ylim([0.0, 1])

# plt.show()
# plt.savefig("bulk2.png")

Fit each dimension separately

In [11]:
plt.cla()

# And then perform estimation using expectation maximization with two basis kernels
em = HawkesEM(40, kernel_size=100, n_threads=8, verbose=False, tol=1e-3)
em.fit(hawkes.timestamps)


fig = plot_hawkes_kernels(em, hawkes=hawkes, show=False)
fig.set_size_inches(15, 10)  # Adjust the size as needed
plt.tight_layout()  # Adjust the layout to prevent overlap
for ax in fig.axes:
    ax.set_ylim([0.0, 0.05])

plt.show()
plt.savefig("separate.png")